In [1]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pylandau
from scipy.optimize import curve_fit
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)

In [ ]:
loadEvents = 100000
runIDs = arr([637,635,639,633,638,636,640,634])

data = {}
for runID in runIDs:
    data[runID] = load_run(runID, loadEvents*4)
    data[runID]["nEvts"] = min(data[runID]["nEvts"],loadEvents)
    
# data[runID]["data"][evt,pixel,M[key]]
M = data[runIDs[0]]["M"]

In [ ]:
binRange = [0.025,0.2]
binN = 40
binWidth = (binRange[1]-binRange[0])/binN
print(binWidth)
key = "AmplitudeEst"
fitRangeBinsLow = arr([
    [27,25,24,28],
    [26,25,23,26],
    [25,23,22,24],
    [24,21,20,23],
    [22,21,17,23],
    [22,20,16,22],
    [21,19,16,21],
    [21,18,15,20],
])
fitRangeBinsHigh = arr([ [35]*4, [34]*4, [34]*4, [34]*4, [34]*4, [34]*4, [34]*4, [38]*4 ])
    

fig, axs = Hist.create_fig(2,4, [10,12])
pixMap = [1,3,0,2]
PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])
    
for i,runID in enumerate(runIDs):
    data[runID]["charge_cal"] = uarr([0]*4, [0]*4)
    for i_pix in range(4):
        entries = data[runID]["data"][:,pixMap[i_pix],M[key]]
        hist, bins = np.histogram(entries, bins=binN, range=binRange)
        
        fitMask = np.zeros(len(bins)-1, dtype=bool)
        for j in range(len(bins)-1):
            if j > fitRangeBinsLow[i,i_pix] and j < fitRangeBinsHigh[i,i_pix]:
                fitMask[j] = True
        p0 = arr([1000, 0.13, 0.01])
        popt, perr, chi2, ndeg = Hist.fit_wrapper(hist, bins, fitMask, fitfunc_gauss, p0)
        data[runID]["charge_cal"][i_pix] = 1620 / uf(popt[1], perr[1])
        
        fitX = np.arange(bins[:-1][fitMask][0]+binWidth/2, bins[:-1][fitMask][-1]+3/2*binWidth, binWidth/20)
        x = np.arange(binRange[0], binRange[1],(binRange[1]-binRange[0])/1000)     
        
        label = PixNames[i_pix]+" N="+str(int(np.sum(hist)/1000))+"k" +"\n  "+r"$\mu$"+"={:.1u}".format(uf(popt[1], perr[1])) +"\n  "+r"chi2red"+"={:.1f} / {:n}".format(chi2,ndeg)
        
        axs[i].plot(fitX, fitfunc_gauss(fitX,*popt), color="black", lw=1, label="")
        axs[i].plot(x, fitfunc_gauss(x,*popt), color="black", lw=.5, ls=":", label="")
        Hist.draw(axs[i], hist, bins, color="C"+str(i_pix), label=label, fill_alpha=0.1)
    axs[i].set_xlim(binRange)

subtitles = dict_to_arr(data, runIDs, "krum_bias_trim")
subtitles = ["krum_bias_trimming = "+str(subtitles[i])+"nA" for i in range(len(subtitles))]
Hist.finalize(data[runID], fig, axs, "Amplitude [V]", "Entries", "Amplitude", subtitles, measurement="Fe55_all", logy=False)
fig.savefig("/home/jona/DESY/analysis_python/output/Amplitude.pdf")

In [ ]:
fig, ax = Hist.create_fig(1,1,[6,4])

runIDs = arr([637,635,639,633,638,636,640,634])
for i_pix in range(4):
    krum_bias_trimS = dict_to_arr(data, runIDs, "krum_bias_trim")
    charge_calS = uarr([val(data[runID]["charge_cal"][i_pix]) for runID in runIDs],[dev(data[runID]["charge_cal"][i_pix]) for runID in runIDs])
    # Hist.draw(axS, val(charge_cal), np.arange(len(runIDs)), color="C"+str(i_pix), label=PixNames[i_pix], fill_alpha=0.1)
    ax.errorbar(krum_bias_trimS, val(charge_calS), yerr=dev(charge_calS), fmt="o-", color="C"+str(i_pix), label=PixNames[i_pix], capsize=3, markersize=3)

ax.set_xticks(krum_bias_trimS)
Hist.finalize(data[runID], fig, ax, "krum_bias_trim [nA]", "Charge [e-]", "Charge Calibration Factors", subtitles, measurement="Fe55_all", logy=False)
fig.savefig("output/ChargeCalibrationFactors.pdf")

In [ ]:
fig, ax = Hist.create_fig(1,1,[6,4])

runIDs = arr([637,635,639,633,638,636,640,634])
for i_pix in range(4):
    krum_bias_trimS = dict_to_arr(data, runIDs, "krum_bias_trim")
    charge_calS = uarr([val(data[runID]["charge_cal"][i_pix]) for runID in runIDs],[dev(data[runID]["charge_cal"][i_pix]) for runID in runIDs])
    charge_calS = 1 / (charge_calS / 1620) * 1000
    ax.errorbar(krum_bias_trimS, val(charge_calS), yerr=dev(charge_calS), fmt="o-", color="C"+str(i_pix), label=PixNames[i_pix], capsize=3, markersize=3)

ax.set_xticks(krum_bias_trimS)
Hist.finalize(data[runID], fig, ax, "krum_bias_trim [nA]", "Fe55 Peak Position [mV]", "Fe55 Peak Position", subtitles, measurement="Fe55_all", logy=False)
fig.savefig("output/Fe55PeakPosition.pdf")

In [ ]:
def get_calibration_factor_(data_, krum_bias_trim, pix):
    runIDs = arr([637,635,639,633,638,636,640,634])
    krum_bias_trims = dict_to_arr(data_, runIDs, "krum_bias_trim")
    runID = runIDs[np.where(krum_bias_trims == krum_bias_trim)[0][0]]
    return data_[runID]["charge_cal"][pix]

def get_calibration_factorS(data_, krum_bias_trimS):
    if len(krum_bias_trimS) != 4:
        print("Error: krum_bias_trimS must have length 4, coresponding to the 4 pixels")
    return arr([get_calibration_factor_(data_, krum_bias_trimS[pix], pix).nominal_value for pix in range(4)])

print("184: ", get_calibration_factorS(data, arr([4.57,6.78,5.53,5.53])))
print("190: ", get_calibration_factorS(data, arr([2.32,3.30,3.30,3.30])))
print("192: ", get_calibration_factorS(data, arr([6.78,9.91,9.91,9.91])))



In [ ]:
charge_cal = {}
for krum in arr([2.32, 3.30, 4.57, 5.53, 6.78, 7.73, 8.96, 9.91]):
    charge_cal[krum] = arr(get_calibration_factorS(data, arr([krum]*4)))
charge_cal